# 實作多個介面的同名方法

In [4]:
interface I龜仙流 { void Attack(); } //龜派氣功
interface I北界王 { void Attack(); } //界王拳

class SuperSaiyanJin : I龜仙流, I北界王 {
	void I龜仙流.Attack() => Console.WriteLine("使用龜派氣功");
	void I北界王.Attack() => Console.WriteLine("使用界王拳");
}
var ssj = new SuperSaiyanJin();
((I龜仙流)ssj).Attack();
((I北界王)ssj).Attack();

使用龜派氣功
使用界王拳


# 測試

In [8]:
public enum EnumAttack {
	None,
	龜派氣功,
	界王拳,	
	沙鍋大的拳頭,
	水之呼吸貳之型水車,
}

public static readonly Dictionary<EnumAttack, (string 技能名稱, int 傷害, double 爆擊機率)> skillInfo 
	= new Dictionary<EnumAttack, (string, int, double)> {
	{ EnumAttack.None, ("None", 0, 0.0) },
	{ EnumAttack.龜派氣功, ("龜派氣功", 30, 0.2) },
	{ EnumAttack.界王拳, ("界王拳", 50, 0.3) },
	{ EnumAttack.沙鍋大的拳頭, ("沙鍋大的拳頭", 10, 0.4) },
	{ EnumAttack.水之呼吸貳之型水車, ("水之呼吸貳之型水車", 20, 0.5) },
};

interface I龜仙流 { AttackInfo Attack(Player p); }
interface I北界王 { AttackInfo Attack(Player p); }
interface I水之呼吸 { AttackInfo Attack(Player p); }

class SuperSaiyanJin : I龜仙流, I北界王, I水之呼吸 {
	public SuperSaiyanJin(string name) { Name = name; }
	public string Name { get; }

	AttackInfo I龜仙流.Attack(Player p) {
		var attack = EnumAttack.龜派氣功;

		var ai = new AttackInfo();		
		ai.Attack = attack;
		ai.AttackTarget = p.Name;		
		Show攻擊資訊(ai, p);	
		return ai;
	}

	AttackInfo I北界王.Attack(Player p) {
		var attack = EnumAttack.界王拳;

		var ai = new AttackInfo();		
		ai.Attack = attack;
		ai.AttackTarget = p.Name;		
		Show攻擊資訊(ai, p);	
		return ai;
	} 

	public AttackInfo Attack(Player p) {
		var attack = EnumAttack.沙鍋大的拳頭;

		var ai = new AttackInfo();		
		ai.Attack = attack;
		ai.AttackTarget = p.Name;		
		Show攻擊資訊(ai, p);	
		return ai;
	} 

	AttackInfo I水之呼吸.Attack(Player p) {
		var attack = EnumAttack.水之呼吸貳之型水車;
		
		var ai = new AttackInfo();		
		ai.Attack = attack;
		ai.AttackTarget = p.Name;		
		Show攻擊資訊(ai, p);	
		return ai;
	}

	private void Show攻擊資訊(AttackInfo ai, Player p)
		=> Console.WriteLine($"{Name}使用 {ai.Attack}({skillInfo[ai.Attack].傷害}) 爆擊機率 {skillInfo[ai.Attack].爆擊機率} => 攻擊 {p.Name}");	
}

//攻擊資訊
public class AttackInfo {
	public string AttackTarget { get; set; }
	public EnumAttack Attack { get; set; }
}

class Player {
	public Player(string name) { Name = name; }
	public string Name { get;}
	public int HP { 
		get { return _hp; }
		// HP 扣到 < 0 時設成 0
    private set { _hp = value < 0 ? 0 : value; }
	}

	private int _hp = 100;
	public void My挨揍(AttackInfo ai) {	
		//
		if(ai.AttackTarget != Name) {
			return;
		}

		if(IsDead()) {
			Console.WriteLine($"不要再打了!不要再打了! {Name} 已經倒了!");
			return;
		}

		Console.WriteLine($"{Name} 站上來挨揍!");

		bool isCritical = (new Random()).NextDouble() < skillInfo[ai.Attack].爆擊機率;
		int damage = skillInfo[ai.Attack].傷害;
		damage = damage * (isCritical ? 2 : 1);

		Console.WriteLine($"爆擊{(isCritical ? "成功" : "失敗")}");

		int beforeHP = HP;
		HP -= damage;
		Console.WriteLine($"{Name}({beforeHP}) 扣 {damage} 滴 HP，剩 {HP} 滴。");

		if (IsDead()) {
			Console.WriteLine($"{Name} 倒地");
		}	
	}

	public bool IsDead() => HP == 0;
}

public class Event挨揍廣播器 {
	public delegate void MyEventHandler(AttackInfo ai);
	public delegate void MyEventHandlerArea();
	public event MyEventHandler 挨揍列隊;

	public void Raise攻擊(AttackInfo ai) {
		挨揍列隊?.Invoke(ai);
		Console.WriteLine();
	}
}

Event挨揍廣播器 挨揍廣播器 = new Event挨揍廣播器();
var sfw1 = new Player("小廢物一號");
var sfw2 = new Player("小廢物二號");
挨揍廣播器.挨揍列隊 += sfw1.My挨揍;
挨揍廣播器.挨揍列隊 += sfw2.My挨揍;

var swk = new SuperSaiyanJin("孫悟空");

挨揍廣播器.Raise攻擊(((I龜仙流)swk).Attack(sfw1));
挨揍廣播器.Raise攻擊(((I北界王)swk).Attack(sfw1));
挨揍廣播器.Raise攻擊(swk.Attack(sfw1));
挨揍廣播器.Raise攻擊(((I水之呼吸)swk).Attack(sfw1));

挨揍廣播器.Raise攻擊(((I龜仙流)swk).Attack(sfw2));
挨揍廣播器.Raise攻擊(((I北界王)swk).Attack(sfw2));
挨揍廣播器.Raise攻擊(swk.Attack(sfw2));
挨揍廣播器.Raise攻擊(((I水之呼吸)swk).Attack(sfw2));

孫悟空使用 龜派氣功(30) 爆擊機率 0.2 => 攻擊 小廢物一號
小廢物一號 站上來挨揍!
爆擊成功
小廢物一號(100) 扣 60 滴 HP，剩 40 滴。

孫悟空使用 界王拳(50) 爆擊機率 0.3 => 攻擊 小廢物一號
小廢物一號 站上來挨揍!
爆擊失敗
小廢物一號(40) 扣 50 滴 HP，剩 0 滴。
小廢物一號 倒地

孫悟空使用 沙鍋大的拳頭(10) 爆擊機率 0.4 => 攻擊 小廢物一號
不要再打了!不要再打了! 小廢物一號 已經倒了!

孫悟空使用 水之呼吸貳之型水車(20) 爆擊機率 0.5 => 攻擊 小廢物一號
不要再打了!不要再打了! 小廢物一號 已經倒了!

孫悟空使用 龜派氣功(30) 爆擊機率 0.2 => 攻擊 小廢物二號
小廢物二號 站上來挨揍!
爆擊成功
小廢物二號(100) 扣 60 滴 HP，剩 40 滴。

孫悟空使用 界王拳(50) 爆擊機率 0.3 => 攻擊 小廢物二號
小廢物二號 站上來挨揍!
爆擊失敗
小廢物二號(40) 扣 50 滴 HP，剩 0 滴。
小廢物二號 倒地

孫悟空使用 沙鍋大的拳頭(10) 爆擊機率 0.4 => 攻擊 小廢物二號
不要再打了!不要再打了! 小廢物二號 已經倒了!

孫悟空使用 水之呼吸貳之型水車(20) 爆擊機率 0.5 => 攻擊 小廢物二號
不要再打了!不要再打了! 小廢物二號 已經倒了!

